<a href="https://colab.research.google.com/github/mateusduarte-max/Projetos_Python/blob/main/11_Reputacao_Nets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install --upgrade openpyxl # atualização pacote excel
!pip install workadays # modulo para calcular data prometida e data limite de postagem
!pip install unidecode # modulo para retirar caracteres especiais

import pandas as pd
import re
import datetime
from workadays import workdays as wd
import numpy as np
import unidecode

# Bibliotecas Google Golab
from google.colab import auth
auth.authenticate_user()
# Importar biblioteca biqquery
from google.cloud import bigquery
# Bibliotecas Google Sheet
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
from gspread_dataframe import set_with_dataframe
# Montar Drive
from google.colab import drive
drive.mount('/content/drive')

# Classe Dados
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_colab.py /content
from modulos_colab import dados
# Chaves e tokens
!cp /content/drive/MyDrive/Colab_Notebooks/Autenticacoes/chaves_tokens.py /content
%run chaves_tokens.py
# Querys reputação
!cp /content/drive/MyDrive/Colab_Notebooks/querys/querys_reputacao.ipynb /content
%run querys_reputacao.ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 4.2 MB/s 
Mounted at /content/drive


querys_reputacao.ipynb:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.


Calculo Reputação

In [ ]:
# Consulta  bigQuery reclamações formais
reclamacoes_formais = dados.consulta_bigquery(projeto_gcp_1, query_reclamacoes_formais)
# Tratativa base
reclamacoes_formais.rename(columns={'seller_id_pedido':'LOJISTA'}, inplace=True)
reclamacoes_formais['LOJISTA'] = reclamacoes_formais['LOJISTA'].str.strip().str.lower().str.split('/',1).str[0].str.split('-', 1).str[0]
reclamacoes_formais_resumo = pd.pivot_table(reclamacoes_formais, index='LOJISTA', values='pedido', aggfunc='nunique', margins=True)
reclamacoes_formais_resumo.rename(columns={'pedido':'Qtd_Reclamacoes_Formais'}, inplace=True)

# Grava na tabela (Google Drive)
dados.grava_tabela_drive('Base_reclamacoes_formais', 'Página1',reclamacoes_formais)


#reclamacoes_formais = busca_tabela_drive('Base_reclamacoes_formais')
#reclamacoes_formais.rename(columns={'seller_id_pedido':'LOJISTA'}, inplace=True)
#reclamacoes_formais['LOJISTA'] = reclamacoes_formais['LOJISTA'].str.strip().str.lower().str.split('/',1).str[0].str.split('-', 1).str[0]
#reclamacoes_formais_resumo = pd.pivot_table(reclamacoes_formais, index='LOJISTA', values='AtendimentoCodigo', aggfunc='count', margins=True)
#reclamacoes_formais_resumo.rename(columns={'AtendimentoCodigo':'Qtd_Reclamacoes_Formais'}, inplace=True)

######################  ############################################################################################################
# Consulta pedidos bigQuery nets
base_pedidos = dados.consulta_bigquery(projeto_gcp_1, query_pedidos_nets)

###################### Retirada pedidos com checkin - Base original #########################################################################################
#pedidos['COD_PEDIDO'] = pedidos['COD_PEDIDO'].astype(str)
#pedidos_checkin = pd.merge(pedidos,pedidos_ckeckin, how='left', on='COD_PEDIDO' )
#base_pedidos = pedidos_checkin.query('STATUS_checkin!="Finalizado"')
###################### Tratativa base original ##############################################################################################################

# Formatar campos de data
base_pedidos['DATA_APROV'] = base_pedidos['DATA_APROV'].astype(object)
base_pedidos['DATA_APROV'] = pd.to_datetime(base_pedidos['DATA_APROV'])
base_pedidos['mes_data_aprovacao'] = base_pedidos['DATA_APROV'].dt.month
base_pedidos['DIAS_PRAZO'] = base_pedidos['DIAS_PRAZO'].replace('',0)
base_pedidos['DATA_PEDIDO'] = base_pedidos['DATA_PEDIDO'].astype(object)
base_pedidos['DATA_PEDIDO'] = pd.to_datetime(base_pedidos['DATA_PEDIDO'])
base_pedidos['MES_DATA_PEDIDO'] = base_pedidos['DATA_PEDIDO'].dt.month
base_pedidos['DATA_ENTREGA'] = pd.to_datetime(base_pedidos['DATA_ENTREGA'])
base_pedidos['DATA_DESPACHO'] = pd.to_datetime(base_pedidos['DATA_DESPACHO'])
base_pedidos['Hoje'] = datetime.datetime.today().strftime('%Y-%m-%d')
base_pedidos['Hoje'] = pd.to_datetime(base_pedidos['Hoje'], format='%Y-%m-%d')
#Formatar colunas Lojista e Id seller
base_pedidos['STATUS'] = base_pedidos['STATUS'].str.strip()
base_pedidos['LOJISTA'] = base_pedidos['LOJISTA'].str.lower()
base_pedidos['ID_SELLER'] = base_pedidos['ID_SELLER'].str.strip()
base_pedidos['ID_SELLER'] = base_pedidos['ID_SELLER'].astype(str)

# Data Prometida
base_pedidos['DATA_PROMETIDA'] = pd.to_datetime(base_pedidos['DATA_PROMETIDA'])
base_pedidos['mes_data_prometida'] = base_pedidos['DATA_PROMETIDA'].dt.month

# Sellers blocado
sellers_blocado = ['13514',
                  '7968',
                  '9321',
                  '3328',
                  '1483',
                  '4761',
                  '6749',
                  '13590',
                  '12806',
                  '4901',
                  '5382',
                  '2281',
                  '12114',
                  '3468',
                  '10641',
                  '4282',
                  '3441',
                  '5021',
                  '5941',
                  '581',
                  '9963',
                  '13268',
                  '4188',
                  '7930',
                  '12666',
                  '12739',
                  '4764',
                  '7861',
                  '12865',
                  '11932',
                  '7644',
                  '12089',
                  '3464',
                  '6721',
                  '13201',
                  '2082',
                  '3421',
                  '12814',
                  '12799',
                  '5601',
                  '7934',
                  '12855',
                  '11956',
                  '12130',
                  '9501',
                  '4230',
                  '12844',
                  '14699',
                  '581',
                  '14673',
                  '14630',
                  '13268',
                  '14278',
                  '14587',
                  '14887',
                  '14550',
                  '14970',
                  '15182',
                  '14897',
                  '15057',
                  '14967',
                  '1356',
                  '3131',
                  '15237',
                  '15173',
                  '13002',
                  '15244',
                  '15116',
                  '15574',
                  '15510',
                  '15269',
                  '14697',
                  '3385',
                  '15812',
                  '16164',
                  '15971',
                  '16243',
                  '15206',
                  '16273',
                  '16116',
                  '14468',
                  '16755',
                  '17115',
                  '14755',
                  '17008',
                  '16151',
                  '17274',
                  '17471']
#Função para coluna blocado
def seller_blocado(base_pedidos):
  if (base_pedidos['ID_SELLER'] in sellers_blocado):
    return 'sim'
  else:
    return 'não' 
# Coluna Blocado
base_pedidos['Blocado'] = base_pedidos.apply(seller_blocado, axis=1)

################################################################# Base pedidos  ###################################################################
base_pedidos_mes_v1 = base_pedidos.query('DATA_PROMETIDA<="{}"'.format(hoje)).drop_duplicates()

base_pedidos_mes_v1['validacao_data'] = base_pedidos_mes_v1['DATA_APROV'].fillna('sem_data')

def excluir_pedidos_cancelados(base_pedidos_mes_v1):
    if base_pedidos_mes_v1['STATUS'] == 'CANCELADO' and base_pedidos_mes_v1['validacao_data']=='sem_data':
        return 'excluir'
    else:
        return 'ok'

base_pedidos_mes_v1['validacao_pedidos'] = base_pedidos_mes_v1.apply(excluir_pedidos_cancelados, axis=1)

base_pedidos_mes = base_pedidos_mes_v1.query('validacao_pedidos!="excluir"')

base_pedidos_mes.drop(columns=["validacao_pedidos", "validacao_data"], inplace=True)

# Total pedidos
base_seller_lojista = base_pedidos_mes[['ID_SELLER','LOJISTA', 'Tipo_Envio']]
base_tt_pedidos = pd.DataFrame(pd.pivot_table(base_pedidos_mes, index=['ID_SELLER','LOJISTA','Tipo_Envio'], columns='STATUS', values='COD_PEDIDO', aggfunc='count',margins=True, margins_name='TT_Pedidos')).fillna(0)

################################################################## Pedidos Entregues ################################################################
# Pedidos Entregues
base_pedidos_entregues = base_pedidos_mes.query('STATUS=="ENTREGUE"')
def analise_pedidos(base_pedidos_entregues):
  if base_pedidos_entregues['DATA_ENTREGA'] <= base_pedidos_entregues['DATA_PROMETIDA']:
    return 'entregue_dentro_prazo'
  else:
    return 'entregue_fora_prazo'
base_pedidos_entregues['status_entrega'] = base_pedidos_entregues.apply(analise_pedidos, axis=1)
resumo_base_pedidos_entregues = pd.DataFrame(pd.pivot_table(base_pedidos_entregues, index=['ID_SELLER','LOJISTA'], columns='status_entrega', values='COD_PEDIDO', aggfunc='count').fillna(0))
##################################################################3 Concatenação 1 ######################################################################

concatenacao_0 = pd.merge(base_seller_lojista,base_tt_pedidos, how='left', on=['ID_SELLER','LOJISTA','Tipo_Envio']).fillna(0).drop_duplicates()
concatenacao_1 = pd.merge(concatenacao_0,resumo_base_pedidos_entregues, how='left' , on=['ID_SELLER','LOJISTA']).drop_duplicates()

##################################################################### Pedidos Despachados ################################################################

# Base pedidos despachado
base_pedidos_despachado = base_pedidos_mes.query('STATUS==["DESPACHADO","ENTREGUE"] & DATA_DESPACHO!=""').drop_duplicates()
# Função limite de postagem
def data_limite_postagem(base_pedidos_despachado):
  if base_pedidos_despachado['Tipo_Envio'] == 'NSE':
    return wd.workdays(base_pedidos_despachado['DATA_APROV'], base_pedidos_despachado['dias_prazo_nse'], country='BR')
  elif base_pedidos_despachado['Tipo_Envio'] == 'Não' and base_pedidos_despachado['Blocado'] == 'não': 
    return wd.workdays(base_pedidos_despachado['DATA_APROV'], 2, country='BR')
  elif base_pedidos_despachado['Tipo_Envio'] == 'Não' and base_pedidos_despachado['Blocado'] == 'sim': 
    return wd.workdays(base_pedidos_despachado['DATA_APROV'], 5, country='BR')  
  else:
      return ""  
base_pedidos_despachado['DATA_LIMITE_POSTAGEM'] = base_pedidos_despachado.apply(data_limite_postagem, axis=1)

# Função análise postagem
def analise_postagem(base_pedidos_despachado):
  if base_pedidos_despachado['DATA_DESPACHO'] <= base_pedidos_despachado['DATA_LIMITE_POSTAGEM']:
    return 'postagem_dentro_prazo'
  else:
    return 'postagem_fora_prazo'
base_pedidos_despachado['status_postagem'] = base_pedidos_despachado.apply(analise_postagem, axis=1)
resumo_base_pedidos_despachado = pd.DataFrame(pd.pivot_table(base_pedidos_despachado, index=['ID_SELLER','LOJISTA'], columns='status_postagem', values='COD_PEDIDO', aggfunc='count').fillna(0))

##################################################################3 Concatenação 2 ####################################################################

concatenacao_2 = pd.merge(concatenacao_1,resumo_base_pedidos_despachado, how='left' , on=['ID_SELLER','LOJISTA']).drop_duplicates()

##################################################################### Pedidos Pendentes ################################################################
# Base Pedidos Faturados
base_pedidos_faturado = base_pedidos_mes.query('STATUS=="FATURADO"')
# Função limite postagem
def data_limite_postagem(base_pedidos_faturado):
  if base_pedidos_faturado['Tipo_Envio'] == 'NSE':
    return wd.workdays(base_pedidos_faturado['DATA_APROV'], base_pedidos_faturado['dias_prazo_nse'], country='BR')
  elif base_pedidos_faturado['Tipo_Envio'] == 'Não' and base_pedidos_faturado['Blocado'] == 'não': 
    return wd.workdays(base_pedidos_faturado['DATA_APROV'], 2, country='BR')
  elif base_pedidos_faturado['Tipo_Envio'] == 'Não' and base_pedidos_faturado['Blocado'] == 'sim': 
    return wd.workdays(base_pedidos_faturado['DATA_APROV'], 5, country='BR')
  else:
      return ""    
base_pedidos_faturado['DATA_LIMITE_POSTAGEM'] = base_pedidos_faturado.apply(data_limite_postagem, axis=1)

# Base Pedidos Pronto para faturar
base_pedidos_pronto_faturar = base_pedidos_mes.query('STATUS=="PRONTO PARA FATURAR" & DATA_APROV!=""')
base_pedidos_pronto_faturar['dias_prazo_nse'] = base_pedidos_pronto_faturar['dias_prazo_nse'].astype(int)
# Função limite postagem
def data_limite_postagem(base_pedidos_pronto_faturar):
  if base_pedidos_pronto_faturar['Tipo_Envio'] == 'NSE':
    return wd.workdays(base_pedidos_pronto_faturar['DATA_APROV'], base_pedidos_pronto_faturar['dias_prazo_nse'], country='BR')
  elif  base_pedidos_pronto_faturar['Tipo_Envio'] == 'Não' and base_pedidos_pronto_faturar['Blocado'] == 'não': 
    return wd.workdays(base_pedidos_pronto_faturar['DATA_APROV'], 2, country='BR')
  elif  base_pedidos_pronto_faturar['Tipo_Envio'] == 'Não' and base_pedidos_pronto_faturar['Blocado'] == 'sim': 
    return wd.workdays(base_pedidos_pronto_faturar['DATA_APROV'], 5, country='BR')  
  else:
      return ""  
base_pedidos_pronto_faturar['DATA_LIMITE_POSTAGEM'] = base_pedidos_pronto_faturar.apply(data_limite_postagem, axis=1)

# Concatenação bases
pedidos_pendentes = pd.concat([base_pedidos_faturado, base_pedidos_pronto_faturar]).drop_duplicates()


# Coluna ontem
pedidos_pendentes['Ontem'] = pedidos_pendentes['Hoje'] - datetime.timedelta(days = 1)
# Coluna Pedidos Atrasados
def pedidos_em_atraso(pedidos_pendentes):
  if pedidos_pendentes['DATA_PROMETIDA'] <= pedidos_pendentes['Ontem']: 
    return 'Pedido_Atrasado'
  else:
    return ''
pedidos_pendentes['Pedidos_Atrasados'] =  pedidos_pendentes.apply(pedidos_em_atraso, axis=1)   

# Coluna Expedição  Atrasada
def expedicao_em_atraso(pedidos_pendentes):
  if pedidos_pendentes['DATA_LIMITE_POSTAGEM'] <= pedidos_pendentes['Ontem']: 
    return 'Expedicao_Atrasada'
  else:
    return ''
pedidos_pendentes['Pedidos_Expedicao_Atrasada'] =  pedidos_pendentes.apply(expedicao_em_atraso, axis=1)

base1 = pedidos_pendentes.query('Pedidos_Atrasados=="Pedido_Atrasado"').drop_duplicates(subset=['ID_SELLER', 'COD_PEDIDO_LOJA'])
resumo_base1 = pd.pivot_table(base1, index='ID_SELLER', values=['Pedidos_Atrasados'], aggfunc='count')
base2 = pedidos_pendentes.query('Pedidos_Expedicao_Atrasada=="Expedicao_Atrasada"').drop_duplicates(subset=['ID_SELLER', 'COD_PEDIDO_LOJA'])
resumo_base2 = pd.pivot_table(base2, index='ID_SELLER', values=['Pedidos_Expedicao_Atrasada'], aggfunc='count')

######################################################################### Dados KA Nets ######################################################################################
dados_ka = dados.consulta_bigquery(projeto_gcp_1, query_ka)
dados_ka.rename(columns={'group_owner':'Assistido', 'code':'ID_SELLER'}, inplace=True)
dados_ka['Assistido'] = dados_ka['Assistido'].str.replace('Não Assistido', 'Não_Assistido')
# Função sellers ka
def sellers_ka(dados_ka):
  if dados_ka['Assistido'] == 'Assistido':
    return "Key Account"
  else:
    return 'Long Tail'
dados_ka['Key Account?']  = dados_ka.apply(sellers_ka, axis=True)
dados_ka_final = dados_ka[['ID_SELLER', 'Key Account?']]
dados_ka_final['ID_SELLER'] = dados_ka_final['ID_SELLER'].apply(str)
dados_ka_final['ID_SELLER'] = dados_ka_final['ID_SELLER'].str.strip()

##################################################################3 Concatenação 3 e 4 ####################################################################################

#concatenacao_3 = pd.merge(concatenacao_2, base_seller_lojista, how='left', on=['ID_SELLER','LOJISTA']).drop_duplicates()
concatenacao_4 = pd.merge(concatenacao_2, dados_ka_final, how='left', on=['ID_SELLER']).drop_duplicates()
######################################################################### Dados CR e Nps #################################################################################

dados_cr_nps = dados.consulta_bigquery(projeto_gcp_1, query_cr_nps)

#dados_cr_nps['CR_Seller'] = round((dados_cr_nps['CR_Seller']*100),2)
dados_cr_nps['CR_Seller'] = pd.to_numeric(dados_cr_nps['CR_Seller']).fillna(0)
dados_cr_nps['NPS_Seller'] = pd.to_numeric(dados_cr_nps['NPS_Seller']).fillna(0)
dados_cr_nps['LOJISTA'] = dados_cr_nps['Seller_id'].str.lower()
dinamica_cr_nps = dados_cr_nps[['LOJISTA', 'CR_Seller', 'NPS_Seller']].drop_duplicates(subset='LOJISTA')

##################################################################3 Concatenação 6 ####################################################################################

concatenacao_5 = pd.merge(concatenacao_4, dinamica_cr_nps , how='left', on=['LOJISTA']).drop_duplicates()
######################################################################### Base Hubsales #################################################################################
# Import base
#'/content/drive/MyDrive/Ciclo Pedidos Nets Novo/sellers Hubsales - BF.xlsx'
base_husales = pd.read_excel(base_hubsales, usecols=['Id Seller', 
                                                    'Nome Loja', 
                                                    'Hubsales?']).fillna(0)
# Ajustes colunas
base_husales.rename(columns={'Id Seller':'ID_SELLER', 'Nome Loja':'LOJISTA'}, inplace=True)
base_husales['ID_SELLER'] = base_husales['ID_SELLER'].apply(int).apply(str)
base_husales['LOJISTA'] = base_husales['LOJISTA'].str.lower()
base_husales_final = base_husales[['ID_SELLER', 'Hubsales?']]

##################################################################3 Concatenação 7 ####################################################################################
concatenacao_6 = pd.merge(concatenacao_5, base_husales_final , how='left', on=['ID_SELLER']).drop_duplicates()
concatenacao_6['Hubsales?'] = concatenacao_6['Hubsales?'].fillna('Não')

######################################################################### Concatenação 8 e 9 #################################################################################

concatencao_7 = pd.merge(concatenacao_6,resumo_base1, how='left', on='ID_SELLER').drop_duplicates()
concatencao_7['Pedidos_Atrasados'] = concatencao_7['Pedidos_Atrasados'].fillna(0)
concatencao_8 = pd.merge(concatencao_7,resumo_base2, how='left', on='ID_SELLER').drop_duplicates()
concatencao_8['Pedidos_Expedicao_Atrasada'] = concatencao_8['Pedidos_Expedicao_Atrasada'].fillna(0)

######################################################################### Base GMV ###################################################################################################
base_pedidos_mes_gmv = pd.pivot_table(base_pedidos_mes, index='ID_SELLER', values='VLR', aggfunc='sum')
base_pedidos_mes_gmv.rename(columns={'VLR':'GMV'}, inplace=True)

######################################################################### Concatenação 10 ###########################################################################################

concatencao_9 = pd.merge(concatencao_8, base_pedidos_mes_gmv, how='left', on='ID_SELLER').drop_duplicates()
concatencao_9['GMV'] = concatencao_9['GMV'].astype(float)
concatencao_9['GMV'] = round(concatencao_9['GMV'],2)

######################################################################### Concatenação 11 ###########################################################################################

concatenacao_10 = pd.merge(concatencao_9, reclamacoes_formais_resumo , how='left', on=['LOJISTA']).drop_duplicates()
concatenacao_10['Qtd_Reclamacoes_Formais'] = concatenacao_10['Qtd_Reclamacoes_Formais'].fillna(0)

######################################################################### Indicador de reputação #####################################################################################
concatenacao_10['% Entrega'] = round((concatenacao_10['entregue_dentro_prazo']/concatenacao_10['TT_Pedidos'])*100,2).fillna(0)
concatenacao_10['% Postagem'] = round((concatenacao_10['postagem_dentro_prazo']/concatenacao_10['TT_Pedidos'])*100,2).fillna(0)
concatenacao_10['% Cancelados'] = round((concatenacao_10['CANCELADO']/concatenacao_10['TT_Pedidos'])*100,2).fillna(0)
concatenacao_10['% Reclamação'] = round((concatenacao_10['Qtd_Reclamacoes_Formais']/concatenacao_10['TT_Pedidos'])*100,2).fillna(0)

# Classificação  regua entrega, despacho, cancelados, reclamação e nps
def regra_entrega(concatenacao_10):
  if concatenacao_10['% Entrega'] >= 0.00 and concatenacao_10['% Entrega'] <= 84.99:
    return 1
  elif concatenacao_10['% Entrega'] >= 85.00 and concatenacao_10['% Entrega'] <= 88.99:
    return 2
  elif concatenacao_10['% Entrega'] >= 89.00 and concatenacao_10['% Entrega'] <= 92.99:
    return 3
  elif concatenacao_10['% Entrega'] >= 93.00 and concatenacao_10['% Entrega'] <= 94.99:
    return 4
  elif concatenacao_10['% Entrega'] >= 95.00:
    return 5
concatenacao_10['reg_entrega'] = concatenacao_10.apply(regra_entrega, axis=1)


def regra_despacho(concatenacao_10):
  if concatenacao_10['% Postagem'] >= 0.00 and concatenacao_10['% Postagem'] <= 84.99:
    return 1
  elif concatenacao_10['% Postagem'] >= 85.00 and concatenacao_10['% Postagem'] <= 88.99:
    return 2
  elif concatenacao_10['% Postagem'] >= 89.00 and concatenacao_10['% Postagem'] <= 92.99:
    return 3
  elif concatenacao_10['% Postagem'] >= 93.00 and concatenacao_10['% Postagem'] <= 94.99:
    return 4
  elif concatenacao_10['% Postagem'] >= 95.00:
    return 5
concatenacao_10['reg_Postagem'] = concatenacao_10.apply(regra_despacho, axis=1)


def regra_cancelados(concatenacao_10):
  if concatenacao_10['% Cancelados'] <= 2.00:
    return 5
  elif concatenacao_10['% Cancelados'] >= 2.01 and concatenacao_10['% Cancelados'] <= 3.00:
    return 4
  elif concatenacao_10['% Cancelados'] >= 3.01 and concatenacao_10['% Cancelados'] <= 4.0:
    return 3
  elif concatenacao_10['% Cancelados'] >= 4.01 and concatenacao_10['% Cancelados'] <= 5.00:
    return 2
  elif concatenacao_10['% Cancelados'] >= 5.01:
    return 1
concatenacao_10['reg_Cancelados'] = concatenacao_10.apply(regra_cancelados, axis=1)


def regra_reclamacao(concatenacao_10):
  if concatenacao_10['% Reclamação'] <= 1.00:
    return 5
  elif concatenacao_10['% Reclamação'] >= 1.01 and concatenacao_10['% Reclamação'] <= 2.50:
    return 4
  elif concatenacao_10['% Reclamação'] >= 2.51 and concatenacao_10['% Reclamação'] <= 3.50:
    return 3
  elif concatenacao_10['% Reclamação'] >= 3.51 and concatenacao_10['% Reclamação'] <= 4.00:
    return 2
  elif concatenacao_10['% Reclamação'] >= 4.01:
    return 1
concatenacao_10['reg_Reclamação'] = concatenacao_10.apply(regra_reclamacao, axis=1)


def regra_nps(concatenacao_10):
  if concatenacao_10['NPS_Seller'] <= 64.99 :
    return 1
  elif concatenacao_10['NPS_Seller'] >= 65.00 and concatenacao_10['NPS_Seller'] <= 67.99:
    return 2
  elif concatenacao_10['NPS_Seller'] >= 68.00 and concatenacao_10['NPS_Seller'] <= 70.00:
    return 3
  elif concatenacao_10['NPS_Seller'] >= 70.01 and concatenacao_10['NPS_Seller'] <= 74.99:
    return 4
  elif concatenacao_10['NPS_Seller'] >= 75.00:
    return 5
concatenacao_10['reg_nps'] = concatenacao_10.apply(regra_nps, axis=1)


def regra_cr(concatenacao_10):
  if concatenacao_10['CR_Seller'] >= 0.301 :
    return 1
  elif concatenacao_10['CR_Seller'] >= 0.201 and concatenacao_10['CR_Seller'] <= 0.300:
    return 2
  elif concatenacao_10['CR_Seller'] >= 0.151 and concatenacao_10['CR_Seller'] <= 0.200:
    return 3
  elif concatenacao_10['CR_Seller'] >= 0.101 and concatenacao_10['CR_Seller'] <= 0.150:
    return 4
  elif concatenacao_10['CR_Seller'] <= 0.100:
    return 5
concatenacao_10['reg_cr'] = concatenacao_10.apply(regra_cr, axis=1)


# Indicador da reputação
def funcao_reputacao(concatenacao_10):
  if concatenacao_10['Tipo_Envio'] == 'Não':
    return round((concatenacao_10['reg_nps']/5*1.45)+(concatenacao_10['reg_entrega']/5*1.50)+(concatenacao_10['reg_cr']/5*0.50)+(concatenacao_10['reg_Reclamação']/5*0.35)+(concatenacao_10['reg_Cancelados']/5*1.20),2)
  elif concatenacao_10['Tipo_Envio'] == 'NSE':
    return round((concatenacao_10['reg_nps']/5*1.45)+(concatenacao_10['reg_Postagem']/5*1.50)+(concatenacao_10['reg_cr']/5*0.50)+(concatenacao_10['reg_Reclamação']/5*0.35)+(concatenacao_10['reg_Cancelados']/5*1.20),2)
concatenacao_10['Indicador_Reputacao'] = concatenacao_10.apply(funcao_reputacao, axis=1)

############################################################################## Ingestão BigQuery ########################################################################################

# Tabela Dash - BigQuery
# renomear colunas para BigQuery

concatenacao_10.rename(columns={'PRONTO PARA FATURAR':'Pronto_para_faturar',
                                'Key Account?':'Key_Account',
                                'Hubsales?':'Hubsales',
                                'Ns_Entregas?':'Ns_Entregas',
                                '% Entrega':'P_Entrega',
                                '% Postagem':'P_Postagem',
                                '% Cancelados':'P_Cancelados',
                                '% Reclamação':'P_Reclamacao',
                                'reg_Reclamação':'reg_Reclamacao'}, inplace=True)

concatenacao_12 = concatenacao_10.query('ID_SELLER!="TT_Pedidos"')
# Gravar tabela
concatenacao_12.to_gbq(destination_table='marketplace_analytics.dados_reputacao_nets',project_id=projeto_gcp_1, if_exists='replace')
# Gravar tabela
dados.grava_tabela_drive('Tb_pedidos_pendentes_reputacao','Página1',pedidos_pendentes)
# Gravar tabela
pedidos_cancelados = base_pedidos_mes.query('STATUS=="CANCELADO"')
dados.grava_tabela_drive('Tb_pedidos_cancelados', 'Página1',pedidos_cancelados)

print("Reputação Atualizada!!")


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/workadays/workdays.py:38: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  while dt_aux in holi

Reputação Atualizada!!


In [ ]:
# Comunicação - Massivo
concatenacao_13 = concatenacao_12.query('TT_Pedidos>=10')
comunicacao = concatenacao_13.copy()

def concat_comunicacao(comunicacao):
    if comunicacao['Tipo_Envio']=='Não':
        return ('Reputação:'+ str(comunicacao['Indicador_Reputacao'])+" =  "+ 
                'Entrega:'+ str(comunicacao['P_Entrega'])+"   "+
                'NPS:'+ str(comunicacao['NPS_Seller'])+"   "+
                'Cancelamento:'+ str(comunicacao['P_Cancelados'])+"   "+
                'CR:'+ str(comunicacao['CR_Seller'])+"   "+
                'Reclamação:'+ str(comunicacao['P_Reclamacao']))
    elif comunicacao['Tipo_Envio']=='NSE':
        return ('Reputação:'+ str(comunicacao['Indicador_Reputacao'])+" =  "+ 
                'Postagem:'+ str(comunicacao['P_Postagem'])+"   "+
                'NPS:'+ str(comunicacao['NPS_Seller'])+"   "+
                'Cancelamento:'+ str(comunicacao['P_Cancelados'])+"   "+
                'CR:'+ str(comunicacao['CR_Seller'])+"   "+
                'Reclamação:'+ str(comunicacao['P_Reclamacao']))
      

comunicacao['Concat_comunicacao'] = comunicacao.apply(concat_comunicacao, axis=1)
comunicacao_v2 = comunicacao[['ID_SELLER', 'LOJISTA', 'Indicador_Reputacao', 'Concat_comunicacao']]
dados.grava_tabela_drive('comunicacao_export', 'Página1',comunicacao_v2)

print(" Tabela Comunicação Atualizada!")


 Tabela Comunicação Atualizada!


In [ ]:
# Atualizar acompanhamento Gerencial
concatenacao_13['Data_Atualizacao'] = datetime.datetime.now().strftime('%Y-%m-%d')
acompanhamento = concatenacao_13[['ID_SELLER',
                                  'LOJISTA', 
                                  'CANCELADO',
                                  'DESPACHADO',
                                  'ENTREGUE',
                                  'FATURADO',
                                  'Pronto_para_faturar',
                                  'TT_Pedidos',
                                  'entregue_dentro_prazo',
                                  'entregue_fora_prazo',
                                  'postagem_dentro_prazo',
                                  'postagem_fora_prazo',
                                  'Key_Account',
                                  'Tipo_Envio',
                                  'CR_Seller',
                                  'NPS_Seller',
                                  'Hubsales',
                                  'Pedidos_Atrasados',
                                  'Pedidos_Expedicao_Atrasada',
                                  'GMV',
                                  'Qtd_Reclamacoes_Formais',
                                  'P_Entrega',
                                  'P_Postagem',
                                  'P_Cancelados',
                                  'P_Reclamacao',
                                  'reg_entrega',
                                  'reg_Postagem',
                                  'reg_Cancelados',
                                  'reg_Reclamacao',
                                  'reg_nps',
                                  'reg_cr',
                                  'Indicador_Reputacao',
                                  'Data_Atualizacao']] 

def linhas():
    arquivo = gc.open('Acompanhamento_Gerencial_Reputação_nets')
    # abrir planilha 'aba'
    aba = arquivo.worksheet('Página1')
    # colocar a aba dentro de um data_frame
    df = pd.DataFrame(aba.get_all_records())
    #Visualização
    numero = df['LOJISTA'].count()+2
    n = int(numero)
    return n

def input_drive():
  #Abrir planilha e primeira aba
    pagina = gc.open('Acompanhamento_Gerencial_Reputação_nets').sheet1
  # Limpar base
  #pagina.clear()
  # Importar dataframe na planilha
    return set_with_dataframe(pagina, acompanhamento, row=linhas(), include_column_header=False)
input_drive()


print("Acompanhamento Gerencial - Reputação Atualizada!!")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Acompanhamento Gerencial - Reputação Atualizada!!
